In [1]:
# 添加搜索路径
import sys
sys.path.insert(0, "/root/mounted/projects/qpair/")

In [2]:
import os
import pickle
import numpy as np
import time, datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, TensorBoard, LambdaCallback

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from config import * # 加载全局配置
from baselines import MODEL_WEM # 加载模型
from baselines import MODEL_WEM_ATTENTION

MODEL_WEIGHTS_FILE = './keras_saved_models/baseline_model_wem_lstm.h5'

# # KN_PAIRS_SAMPLED_PATH = '/root/mounted/datasets/make_data/intermediate/'
# KN_PAIRS_SAMPLED_PATH = '/root/mounted/datasets/data_0521/intermediate/'
KN_PAIRS_SAMPLED_PATH = '/root/mounted/datasets/data_0523/publish/'
# KN_PAIRS_SAMPLED_WIDS_CIDS_TRAIN_PKL = os.path.join(KN_PAIRS_SAMPLED_PATH, 'kn_train_test_pairs_sampled_wids_cids_train.pkl')
# KN_PAIRS_SAMPLED_WIDS_CIDS_TEST_PKL = os.path.join(KN_PAIRS_SAMPLED_PATH, 'kn_train_test_pairs_sampled_wids_cids_test.pkl')
KN_PAIRS_SAMPLED_WIDS_CIDS_TRAIN_PKL = os.path.join(KN_PAIRS_SAMPLED_PATH, 'train.pkl')
KN_PAIRS_SAMPLED_WIDS_CIDS_TEST_PKL = os.path.join(KN_PAIRS_SAMPLED_PATH, 'test.pkl')

TEST_LABEL_CSV = os.path.join(KN_PAIRS_SAMPLED_PATH, '../test_label.csv')

VALIDATION_SPLIT = 0.1
RNG_SEED = 13371447
NB_EPOCHS = 50
BATCH_SIZE = 1024

In [4]:
# 加载模型
# model = MODEL_WEM()
model = MODEL_WEM_ATTENTION()

In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 39)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 39)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 39, 300)      6267600     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 39, 300)      6267600     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectio

In [6]:
# 加载数据集，并切分成train 和 validation
with open(KN_PAIRS_SAMPLED_WIDS_CIDS_TRAIN_PKL, 'rb') as f:
    kn_train_wids_cids_padded = pickle.load(f)

kn_train_wids_cids_padded.head()

,label,qid1,qid2,words1,words2,chars1,chars2
0,1,Q173098,Q208402,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,Q322606,Q105226,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1,Q168520,Q392700,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1,Q269823,Q353449,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1,Q287705,Q280416,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
with open(KN_PAIRS_SAMPLED_WIDS_CIDS_TEST_PKL, 'rb') as f:
    kn_test_wids_cids_padded = pickle.load(f)

kn_test_wids_cids_padded

,qid1,qid2,words1,words2,chars1,chars2
0,Q151557,Q293789,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Q312362,Q489267,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Q183757,Q521124,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Q569210,Q202128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Q471665,Q658625,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,Q559930,Q353992,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,Q144120,Q142113,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,Q480774,Q591009,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,Q475024,Q591843,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,Q278079,Q384012,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
test_label = pd.read_csv(TEST_LABEL_CSV)
test_label

,y_true,is_preliminary
0,1,0
1,1,1
2,1,1
3,1,0
4,1,0
5,1,1
6,1,0
7,1,1
8,1,1
9,1,0


In [9]:
# 暂时的
# kn_train_wids_cids_padded = pd.concat([kn_train_wids_cids_padded, kn_test_wids_cids_padded], ignore_index=True)

In [10]:
q1_inputs = kn_train_wids_cids_padded['words1'].values
q2_inputs = kn_train_wids_cids_padded['words2'].values
inputs = np.stack((q1_inputs, q2_inputs), axis=1)
labels = kn_train_wids_cids_padded['label']
inputs_train, inputs_val, labels_train, labels_val = train_test_split(inputs, labels, test_size=VALIDATION_SPLIT, random_state=RNG_SEED)
q1_train = np.array(list(inputs_train[:,0]))
q2_train = np.array(list(inputs_train[:,1]))
q1_val = np.array(list(inputs_val[:,0]))
q2_val = np.array(list(inputs_val[:,1]))

In [11]:
inputs_train.shape, inputs_val.shape

((187220, 2), (20803, 2))

In [12]:
np.array(list(q1_train)).shape

(187220, 39)

In [13]:
q1_test_inputs = np.array(list(kn_test_wids_cids_padded['words1'].values))
q2_test_inputs = np.array(list(kn_test_wids_cids_padded['words2'].values))
# labels_test = kn_test_wids_cids_padded['label']
labels_test = test_label.y_true

In [ ]:
test_score = []
lambda_cb = LambdaCallback(on_epoch_end= lambda epock, logs: test_score.append(model.evaluate([q1_test_inputs, q2_test_inputs], labels_test, verbose=0)))

print("Starting training at", datetime.datetime.now())
t0 = time.time()
callbacks = [ModelCheckpoint(MODEL_WEIGHTS_FILE, monitor='val_acc', save_best_only=True),TensorBoard(log_dir='/root/mounted/tensorboard/log'), lambda_cb]
history = model.fit([q1_train, q2_train],
                    labels_train,
                    epochs=NB_EPOCHS,
#                     validation_data = ([q1_test_inputs, q2_test_inputs], labels_test),
                    validation_data = ([q1_val, q2_val], labels_val),
                    verbose=1,
                    batch_size=BATCH_SIZE,
                    callbacks=callbacks)
t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

Starting training at 2018-05-25 10:02:05.631794
Train on 187220 samples, validate on 20803 samples
Epoch 1/50
187220/187220 [==============================] - 18s 97us/step - loss: 0.4766 - acc: 0.7733 - val_loss: 0.3047 - val_acc: 0.8721
Epoch 2/50
187220/187220 [==============================] - 17s 89us/step - loss: 0.3094 - acc: 0.8804 - val_loss: 0.2621 - val_acc: 0.8918
Epoch 3/50
187220/187220 [==============================] - 17s 89us/step - loss: 0.2574 - acc: 0.9015 - val_loss: 0.2414 - val_acc: 0.9017
Epoch 4/50
187220/187220 [==============================] - 17s 89us/step - loss: 0.2290 - acc: 0.9127 - val_loss: 0.2354 - val_acc: 0.9049
Epoch 5/50
187220/187220 [==============================] - 17s 89us/step - loss: 0.2064 - acc: 0.9219 - val_loss: 0.2272 - val_acc: 0.9110
Epoch 6/50
187220/187220 [==============================] - 17s 90us/step - loss: 0.1915 - acc: 0.9274 - val_loss: 0.2305 - val_acc: 0.9111


In [ ]:
acc = pd.DataFrame({'epoch': [ i + 1 for i in history.epoch ],
                    'training': history.history['loss'],
                    'validation': history.history['val_loss'],
                    'test': np.array(test_score)[:,0]})
ax = acc.iloc[:,:].plot(x='epoch', figsize=(8,5), grid=True)
ax.set_ylabel("logloss")
ax.set_ylim([0,0.5]);

In [ ]:
np.array(test_score)[:,0], len(test_score)

In [ ]:
# TEST_WIDS_CIDS_PADDED_PKL = os.path.join(INTERMEDIATE_DATA_PATH, 'test_clean_wids_cids_padded.pkl')
# with open(TEST_WIDS_CIDS_PADDED_PKL, 'rb') as f:
#     test_wids_cids_padded = pickle.load(f)
    
# test_wids_cids_padded

In [ ]:
# q1_test_inputs = np.array(list(test_wids_cids_padded['words1'].values))
# q2_test_inputs = np.array(list(test_wids_cids_padded['words2'].values))
# test_labels = test_wids_cids_padded['label']
# q1_test_inputs.shape

In [ ]:
# 直接用最后一个epoch的模型
model.load_weights(MODEL_WEIGHTS_FILE)
test_pred1 = model.predict([q1_test_inputs, q2_test_inputs], batch_size=BATCH_SIZE)
test_pred1 = test_pred1.squeeze()
test_pred1, test_pred1.shape

In [ ]:
test_pred1_df = pd.DataFrame({'y_pre':test_pred1})
test_pred1_df

In [ ]:
TEST_PRED_1 = './predicts/test_pred1.csv'
test_pred1_df.to_csv(TEST_PRED_1, index=False)

In [ ]:
model.load_weights(MODEL_WEIGHTS_FILE)
test_pred2 = model.predict([q1_test_inputs, q2_test_inputs], batch_size=BATCH_SIZE)
test_pred2 = test_pred2.squeeze()
test_pred2

In [ ]:
score = model.evaluate([q1_test_inputs, q2_test_inputs], labels_test)
score

In [ ]:
from sklearn.metrics import log_loss
logloss = log_loss(labels_test, test_pred1_df.y_pre, eps=1e-15)
logloss